In [1]:
import os
import numpy as np
import pandas as pd

%matplotlib inline

stop_words_path = 'stopwords/baidu_stopwords.txt'
stop_words_dict = []

with open(stop_words_path, errors='ignore') as f:
    for line in f.readlines():
        stop_words_dict.append(line.strip())
print(len(stop_words_dict))

1396


In [2]:
import re
import jieba

pttn = re.compile(r'[\s\d\<\>\-\*《》#@￥%$_——!！]+')

def cut_word(word):
    words_list = jieba.lcut(word)
    return ' '.join([re.sub(pttn, "", w) for w in words_list if w not in stop_words_dict and len(w) > 0])

def cut_danmu_type(path):
    video_danmu = []
    video_name = []
    for file in os.listdir(path):
        try:
            f = open(os.path.join(path, file),encoding='utf-8')
            df = pd.read_csv(f, header=0)
        except Exception as e:
            print(f"文件读取失败：{file}")
        try:
            df['cut_word'] = df['danmu'].apply(cut_word)
            video_danmu.append(df)
            video_name.append(file.replace('_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv', ''))
        except Exception as e:
            print(f"分词失败{file}")
        f.close()
    return video_danmu, video_name

digital, dname = cut_danmu_type('digital/')
food, fname = cut_danmu_type('food/')
view, vname = cut_danmu_type('view/')
cinephile, cname = cut_danmu_type('cinephile/')
news, nname = cut_danmu_type('news/')
# danmu_all, all_name = cut_danmu_type('danmu_all/')
digital


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\83545\AppData\Local\Temp\jieba.cache
Loading model cost 0.864 seconds.
Prefix dict has been built successfully.


分词失败农  鸣   芒   果_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv
分词失败【半佛】如何把投资人当韭菜割_哔哩哔哩 (゜-゜)つロ 干杯~-bilibili.csv


[                                           danmu  \
 0                                           物理挂啊   
 1                                          还要靠售价   
 2                                        这个感觉太爽了   
 3                                       安卓平板用户暗喜   
 4                           同样的价格是买一个折叠屏还是买两个手机？   
 5                                            用心了   
 6                                       爱死你每个视频了   
 7                                   这个挖槽太冒犯了 - -   
 8                                       好拙劣的对比。。   
 9                                           有意思了   
 10                                         这个真的强   
 11                                         这个太爽了   
 12                                         这个太爽了   
 13                               这个直接发送，不是输入再点发送   
 14                       miui10.1的都可以这么玩，刚刚试了试超爽   
 15                                      有钱真好，嘿嘿嘿   
 16                                          好方便啊   
 17                        三国杀和ygo没有适配，展开后会出问题

In [ ]:
# 读取情感词典

In [6]:
boson = pd.read_table('BosonNLP_sentiment_score/BosonNLP_sentiment_score.txt', header=None, encoding='utf-8', sep=' ')

c:\users\83545\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [12]:
boson.columns = ['word', 'score']
boson.head()

,word,score
0,最尼玛,-6.704000
1,扰民,-6.497564
2,fuck...,-6.329634
3,RNM,-6.218613
4,wcnmlgb,-5.967100


In [17]:
key = list(boson['word'].values)
boson_dict = dict(zip(list(boson['word'].values), list(boson['score'].values)))
boson_dict

{'最尼玛': -6.7040001263699995,
 '扰民': -6.497564458669999,
 'fuck...': -6.3296339043300005,
 'RNM': -6.21861284426,
 'wcnmlgb': -5.96710044003,
 '2.5:': -5.90459648251,
 'Fxxk': -5.87247473641,
 'MLP': -5.87247473641,
 '吃哑巴亏': -5.77120419579,
 'IAQI': -5.771078371230001,
 'MLGBD': -5.694081915009999,
 'NNND': -5.66228462641,
 'MLGB.': -5.6045774358300005,
 '成甘': -5.6045774358300005,
 '最桑': -5.6045774358300005,
 '真无语': -5.6045774358300005,
 'ＴＭ': -5.6045774358300005,
 '次奥次奥次奥': -5.59258287133,
 'cnmd': -5.54446545761,
 'MBD': -5.502801098430001,
 'NNDX': -5.48173951768,
 '水蛭': -5.48173951768,
 '美素丽': -5.48173951768,
 '草尼': -5.48173951768,
 '凌迟': -5.46005372985,
 '尼玛尼玛尼玛': -5.42622557462,
 '冠周炎': -5.41616190446,
 '加塞儿': -5.41616190446,
 '日尼玛': -5.41616190446,
 '朝仲': -5.41616190446,
 'MD...': -5.3474609293699995,
 'shit....': -5.3474609293699995,
 '人清': -5.3474609293699995,
 '风疹块': -5.3474609293699995,
 'ＴＭＤ': -5.3474609293699995,
 '真桑心': -5.304404434019999,
 'NND...': -5.27532412618,
 '延机':

In [62]:
neg = re.compile(r'(不)|(没)')

def get_danmu_score(danmu):
    score = 0
    flag = 1
    for word in danmu:
        if word in key:
            score += boson_dict.get(word)*flag
        if flag < 1:
            flag = 1
        if len(re.findall(neg, word))>0:
            flag = -1
    return score

In [63]:
def get_video_score(video_list):
    for video in video_list:
        video['score'] = video['cut_word'].map(lambda x: get_danmu_score(x))
    return video_list

get_video_score(digital[:10])

[                                           danmu  \
 0                                           物理挂啊   
 1                                          还要靠售价   
 2                                        这个感觉太爽了   
 3                                       安卓平板用户暗喜   
 4                           同样的价格是买一个折叠屏还是买两个手机？   
 5                                            用心了   
 6                                       爱死你每个视频了   
 7                                   这个挖槽太冒犯了 - -   
 8                                       好拙劣的对比。。   
 9                                           有意思了   
 10                                         这个真的强   
 11                                         这个太爽了   
 12                                         这个太爽了   
 13                               这个直接发送，不是输入再点发送   
 14                       miui10.1的都可以这么玩，刚刚试了试超爽   
 15                                      有钱真好，嘿嘿嘿   
 16                                          好方便啊   
 17                        三国杀和ygo没有适配，展开后会出问题

In [64]:
digital[1]

,danmu,cut_word,score
0,紧急制动hhhhhh,紧急制动 hhhhhh,2.197595
1,极致操作,极致 操作,-1.687964
2,哈哈哈哈哈哈,哈哈哈 哈哈哈,17.264362
3,真的就梦回2G时代,真的 就 梦回 G 时代,-0.029530
4,真实,真实,2.127366
5,令人窒息,令人窒息,0.033617
6,笑死我了,笑 死 我 了,0.124824
7,真的真的ex,真的 真的,3.248399
8,硬盘也没拿个高的存储速度,硬盘 也 没拿个 高 的 存储 速度,1.177933
9,离合松的太快，熄火了,离合 松 的 太快 ， 熄火 了,-0.118894


In [65]:
get_video_score(food[10:30])

[                       danmu                              cut_word      score
 0                        上菜了                                  上菜 了   1.446764
 1                         卧槽                                    卧槽  -1.900114
 2                  前面卤肉饭的等等我                         前面 卤肉饭 的 等等 我  -0.514385
 3                  哈哈哈哈哈哈哈哈哈                           哈哈哈 哈哈哈 哈哈哈  25.896543
 4                        ？？？                                 ？ ？ ？  -2.406471
 5                    这个龙太可爱了                            这个 龙太 可爱 了   1.235005
 6                 怎么没看见金光环绕？                       怎么 没 看见 金光 环绕 ？  -0.959941
 7                        wow                                   wow   2.544586
 8                 哈哈哈哈哈哈哈哈哈哈                          哈哈哈哈 哈哈哈 哈哈哈  28.773937
 9                     凑七个会怎样                             凑 七个 会 怎样  -3.566081
 10              ohhhhhhhhhhh                          ohhhhhhhhhhh   9.855899
 11                       ？？？                       

In [66]:
get_video_score(news[:10])

[                  danmu                    cut_word      score
 0            这是要下台的节奏了？           这 是 要 下台 的 节奏 了 ？  -0.037560
 1     不会拜登周围的人全是FBL的演员吧  不会 拜登 周围 的 人 全是 FBL 的 演员 吧   2.914522
 2             你不当政你当然开心              你 不 当政 你 当然 开心   1.092366
 3                都是为了选票                   都 是 为了 选票   4.872768
 4             我觉得拜登也不咋地             我 觉得 拜登 也 不 咋 地   0.425074
 5                 川普很难啊                    川普 很 难 啊  -0.711558
 6                  啧啧啧～                      啧啧 啧 ～  -0.350722
 7               川普同志辛苦了                  川普 同志 辛苦 了   2.600224
 8                无语。没人？                 无 语 。 没 人 ？  -0.484165
 9                又变成党争了                   又 变成 党争 了  -0.697619
 10                   可怜                          可怜  -0.096493
 11                拜登搞的+                    拜登 搞 的 +   3.043550
 12           拜登赢定的，会拉人心             拜登 赢定 的 ， 会拉 人心   4.274908
 13           得了吧拜登能救你们？              得了吧 拜登 能救 你们 ？   3.750868
 14                 哈哈哈哈                

In [67]:
get_video_score(view[:10])

[                          danmu                                  cut_word  \
 0                          胆大心细                                      胆大心细   
 1                            卧槽                                        卧槽   
 2                        过来刷第二遍                                  过来 刷 第二遍   
 3                            感动                                        感动   
 4                      成功用什么定义？                              成功 用 什么 定义 ？   
 5                        非常diao                                   非常 diao   
 6                     40卧槽，看不出来                                 卧槽 ， 看不出来   
 7                    我们的目标是星辰大海                           我们 的 目标 是 星辰 大海   
 8                  热爱钱不就去砸银行了吗？                     热爱 钱 不 就 去 砸 银行 了 吗 ？   
 9                  不看就左上角 别刷存在感                         不看 就 左上角  别刷 存在 感   
 10                      暗示我的股票？                               暗示 我 的 股票 ？   
 11                      然后被骗hhh                                

In [37]:
get_video_score(cinephile[:10])

[                 danmu                        cut_word      score
 0                 菲利普？                           菲利普 ？   3.440860
 1         小10小11可太可爱啦！                  小  小  可太 可爱 啦    7.041358
 2                  这差距                            这 差距  -1.784429
 3          我的比你粗，我的比你长           我 的 比 你 粗 ， 我 的 比 你 长   0.684087
 4                我要进去了                         我要 进去 了  -1.147362
 5                哈哈哈哈哈                          哈哈哈 哈哈  14.386969
 6     我的可以更大，额。。我的可以变长    我 的 可以 更大 ， 额 。 。 我 的 可以 变 长  -0.883826
 7                 好像憨豆                           好像 憨豆   2.817864
 8     what?what!?what!                                   0.000000
 9              哈哈哈哈哈哈哈                        哈哈哈哈 哈哈哈  20.141756
 10                  魔杖                              魔杖   1.155171
 11               阿瓦达索命                         阿瓦 达索 命  -0.405389
 12          这里真的好可爱www                  这里 真的 好 可爱 www   8.909943
 13         Lovely 哈哈哈哈                    Lovely  哈哈哈哈  13.40

In [68]:
def sum_up(df_list, video_list):
    score = []
    for i in range(len(video_list)):
        score.append(np.mean(df_list[i]['score'].values, axis=0))
    df = pd.DataFrame(score, index=video_list)
    df.columns = ['平均分']
    return df

In [69]:
food_df = sum_up(food[:10], fname[:10])

In [70]:
food_df

,平均分
3分钟就能在宿舍做好的提拉米苏！不用烤箱、不用烹饪的意大利经典甜品和B站奖牌的最佳搭配,3.672415
B站迄今最详细的烤鱼教程！！！Up主耗时3天，呕心沥血,0.267135
Up主做土家酱香饼，一口下去外酥里软、酱香浓郁！！！以后再也不用出去买了,0.990183
Up主做长沙臭豆腐，成本1毛一块，一口下去瞬间爆浆,2.930934
Up主历时30天，终极还原KFC嫩牛五方！！！,0.081053
[定格动画]日本超流畅便当制作,0.299052
《 这才叫 水 视 频！》 创意短片,0.949943
【omozoc原创】日本社畜的绝望寿司,1.470911
【omozoc原创】樱花手握寿司,3.563780
【原木树脂】一枚凤巢龙蛋的诞生过程,1.105181


In [71]:
news_df = sum_up(news[:10], nname[:10])
news_df

,平均分
#特朗普被曝狼狈躲进地下# # 拜登上街与示威者愉快合影#,1.903599
140秒了解美国暴力示威始末,1.476716
2分钟看清：同一时间，中美都在干什么？,4.106347
“你是城管？你叫我去摆摊？你骗我吧？”,4.499780
“围攻”白宫第二日：示威者拆围栏踩警车 国民警卫队出动,2.657421
“宝贝！中国报道我了！”,9.595246
“我是公众人物，你完了！”,1.214288
【中美对比】中国战“疫”时，美国在做什么？,6.184964
【小崔每日秀】5月30日 小崔对最近黑人被警察暴力执法事件发声 美国种族分裂走向极端,0.888342
【崔娃】美国社会对黑人的歧视，黑人女生在教室睡觉被报警，黑人在星巴克不买东西就想用厕所被捕，等其他歧视事件,9.022630


In [72]:
digital_df = sum_up(digital[:10], dname[:10])
digital_df

,平均分
1+1 能否大于 2 —— Mate Xs 长期体验,0.967503
5G终究玩不过百度云,4.543855
iPhone 11体验：和iPhone 11 Pro的差距只有两个字！,1.245951
iPhone老用户使用4个月华为mate30 pro的简单感受分享,1.817428
P40 VS P40pro VS Mate30pro怎么选？,2.237378
p40pro拍照真差,-0.809206
P40发布之后，Mate30 Pro变的更香了？年度真香旗舰,1.316069
「小白测评」 手握一套房！ 华为MateX体验测评 对比三星Fold,0.568751
「小白测评」vivo X50 Pro体验 把云台装手机里是什么体验？,1.168653
「小白测评」两万块的开箱 三星Galaxy Z Flip Thom Browne版 听~是RMB的味道~,1.362275


In [73]:
cinephile_df = sum_up(cinephile[:10], cname[:10])
cinephile_df

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


,平均分
[DW]两位博士见面后竟开始。。。比较音速起子,6.386117
【刘哔】超长版解说！温情解说《汉尼拔》第一季,0.773764
"【史蒂芬周】面对可怖的低语者，神奇女孩做出伟大牺牲,一场绝美的时间之战！《神秘博士》解说+解析SE07-13。",-0.360469
【培根悖论】二十年后重温这部影史前三的系列电影开篇作，依旧回味无穷《哈利波特与魔法石》,2.660072
【培根悖论】垂直结构监狱600多人疯抢一桌饭，《饥饿站台》见证人性的无限坠落,1.055433
【培根悖论】女巫开药带队，蛇王连刀5人！这才是打开《哈利波特与密室》的正确姿势！,4.606761
【木鱼微剧场】《三傻大闹宝莱坞》阿米尔汗主演,1.257964
【木鱼微剧场】《上帝之城》暴力的循环,0.836136
【木鱼微剧场】《低俗小说》,2.855923
【木鱼微剧场】《傲慢与偏见》,3.512223


In [74]:
view_df = sum_up(view[:10], vname[:10])
view_df

,平均分
2020失业潮，普通人能否出奇制胜？,0.397530
【专业性悖论】半佛仙人为何在知乎遭遇大量差评？,-0.438859
【半佛】5本让你怀疑世界的好书,0.971912
【半佛】PUA看似是收割女性，其实是收割男性的钱包！,2.434155
【半佛】不涨价的小米，没有未来,2.864010
【半佛】为什么公司要对员工工资进行保密？,0.403343
【半佛】为什么渣女总是屡屡得手？,2.143579
【半佛】为什么渣男总是无往不利？,1.307752
【半佛】为何川普漠视美国人命,2.910388
【半佛】买不起奢侈品就是穷人吗？不，是聪明人,1.777635


In [75]:
dm = np.mean(digital_df['平均分'].values, axis=0)
nm = np.mean(news_df['平均分'].values, axis=0)
cm = np.mean(cinephile_df['平均分'].values, axis=0)
vm = np.mean(view_df['平均分'].values, axis=0)
fm = np.mean(food_df['平均分'].values, axis=0)

In [76]:
result = pd.DataFrame([dm, fm, vm, cm, nm], index=['数码','美食','观点','影评','新闻'])
result.columns = ['平均分']
result

,平均分
数码,1.441866
美食,1.533059
观点,1.477145
影评,2.358392
新闻,4.154933
